In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import *
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import os
os.makedirs('result', exist_ok=True)  # 创建result文件夹
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings("ignore") #指定忽略警告消息
plt.rcParams['font.sans-serif'] = ['SimHei'] # 指定默认字体
plt.rcParams['axes.unicode_minus'] = False # 解决保存图像是负号'-'显示为方块的问题
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error,mean_absolute_error
from scipy.stats import mstats
from keras.optimizers import Adam


seed=42
# Set the random seed for reproducibility
np.random.seed(seed)  # For NumPy
import random
random.seed(seed)  # For Python's built-in random module

# For TensorFlow/Keras
import tensorflow as tf
tf.random.set_seed(seed)  # For TensorFlow/Keras


2025-02-21 14:57:15.917755: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-21 14:57:15.928892: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-21 14:57:15.932306: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-21 14:57:15.941680: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:

x_data = pd.read_table('/home/rliuaj/balance_sheet/df_X',sep=',',index_col=0)
x_data = x_data.drop(['Market Cap', 'Industry', 'City'],axis=1)
y_data = pd.read_table('/home/rliuaj/balance_sheet/df_y',sep=',',index_col=0)
x_data = x_data.interpolate()
y_data = y_data.interpolate()

In [ ]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=seed) # 20

In [4]:
y_test['new_stock'].value_counts()

new_stock
GRC      3
VIZIO    3
KIMCO    3
CBRE     3
MAA      3
        ..
CENTA    1
ROVR     1
SAGE     1
AIG      1
TEL      1
Name: count, Length: 627, dtype: int64

In [6]:
y_test

,Unnamed: 0,Current Assets,Total Non Current Assets,Current Liabilities,Total Non Current Liabilities Net Minority Interest,Stockholders Equity,new_stock
1078,2023-12-31,2.811350e+09,1.115625e+10,2.013347e+09,5.752349e+09,6.149001e+09,ZS
1957,2021-12-31,1.082797e+07,2.371896e+06,6.039311e+06,2.962711e+06,4.197847e+06,APTIV
2694,2023-06-30,1.646807e+09,1.096525e+09,5.403440e+08,7.445510e+08,1.458437e+09,BANKS
2002,2021-12-31,8.972340e+08,5.069280e+08,3.782450e+08,6.362600e+07,9.622910e+08,ENVX
1213,2023-12-31,1.926967e+09,2.273419e+10,2.889347e+09,1.548694e+10,6.177664e+09,UHS
...,...,...,...,...,...,...,...
3129,2023-12-31,4.446971e+09,2.162750e+09,3.518355e+09,6.205510e+08,2.469778e+09,OSCAR
2287,2022-10-31,6.446520e+08,6.711780e+08,1.937970e+08,5.994400e+07,8.315270e+08,ACC
2567,2024-09-30,1.200544e+09,2.930964e+09,8.075290e+08,1.481164e+09,1.842815e+09,SEE
700,2024-05-31,3.185207e+09,5.983610e+09,1.828519e+09,3.023926e+09,4.316372e+09,TEL


In [39]:
pred=pd.read_csv('result/Transformer_y_pred.csv')
pred.index=y_test.index
pred.columns=y_test.columns[:-1]
pred_concat=pd.read_csv('result/Transformer_y_pred_concat.csv')
pred_concat.index=y_test.index
pred_concat.columns=y_test.columns[:-1]

In [10]:
pred.head()

,Unnamed: 0,Current Assets,Total Non Current Assets,Current Liabilities,Total Non Current Liabilities Net Minority Interest,Stockholders Equity
1078,0,3.266728e+09,9.541004e+09,2.729232e+09,5.612189e+09,4.317826e+09
1957,1,2.530697e+08,7.209217e+08,2.113218e+08,4.276749e+08,3.246753e+08
2694,2,5.849562e+08,1.708467e+09,4.887083e+08,1.004947e+09,7.731804e+08
2002,3,3.016092e+08,8.809089e+08,2.519835e+08,5.181675e+08,3.986579e+08
1213,4,5.943502e+09,1.735911e+10,4.965553e+09,1.021096e+10,7.855934e+09


In [40]:
pred.index=y_test.index
pred_concat.index=y_test.index

In [65]:
mape = np.mean(np.abs((y_test.iloc[:,1:-1] - pred.iloc[:,1:]) / y_test.iloc[:,1:-1]), axis=1)
mape

1078      0.196900
1957    115.800388
2694      0.423604
2002      1.893015
1213      0.730345
           ...    
3129      0.905320
2287      2.154061
2567      0.159529
700       0.197094
839       1.068626
Length: 764, dtype: float64

In [96]:
name='EQRX'
y_test.loc[y_test['new_stock']==name]

,Unnamed: 0,Current Assets,Total Non Current Assets,Current Liabilities,Total Non Current Liabilities Net Minority Interest,Stockholders Equity,new_stock
2014,2021-12-31,1.171790e+09,3.814473e+09,941611000.0,2.060976e+09,1.983676e+09,EQRX


In [97]:
li=y_test.loc[y_test['new_stock']==name].index
pred.loc[li,:]

,Unnamed: 0,Current Assets,Total Non Current Assets,Current Liabilities,Total Non Current Liabilities Net Minority Interest,Stockholders Equity
2014,95,1.249237e+09,3.648690e+09,1.043688e+09,2.146240e+09,1.651214e+09


In [66]:
mape[2014]

0.08538616412196702

In [63]:
r=pred.loc[li,:].iloc[:,3:].values.sum()/pred.loc[li,:].iloc[:,1:3].values.sum()
r

0.9884061860858496

In [64]:
pred.loc[li,:].iloc[:,1:3]*r

,Current Assets,Total Non Current Assets
2014,1.234754e+09,3.606388e+09


In [80]:
merged_result = pd.concat([pred.loc[li,:].iloc[:,1:3]*r, pred.loc[li, :].iloc[:, 3:]], axis=1)
merged_result

,Current Assets,Total Non Current Assets,Current Liabilities,Total Non Current Liabilities Net Minority Interest,Stockholders Equity
2014,1.234754e+09,3.606388e+09,1.043688e+09,2.146240e+09,1.651214e+09


In [82]:
mape_new = np.mean(np.abs((y_test.iloc[:,1:-1] - merged_result) / y_test.iloc[:,1:-1]), axis=1)
mape_new[2014]

0.08513213332917313

In [98]:
li=y_test.loc[y_test['new_stock']==name].index
pred_concat.loc[li,:]

,Unnamed: 0,Current Assets,Total Non Current Assets,Current Liabilities,Total Non Current Liabilities Net Minority Interest,Stockholders Equity
2014,95,1.264398e+09,3.572197e+09,1.032301e+09,2.110884e+09,1.647570e+09


In [99]:
mape_concat[2014]

0.08650243456405322

In [100]:
r=pred_concat.loc[li,:].iloc[:,3:].values.sum()/pred_concat.loc[li,:].iloc[:,1:3].values.sum()
r

0.9905221754039324

In [101]:
merged_result = pd.concat([pred_concat.loc[li,:].iloc[:,1:3]*r, pred_concat.loc[li, :].iloc[:, 3:]], axis=1)
merged_result

,Current Assets,Total Non Current Assets,Current Liabilities,Total Non Current Liabilities Net Minority Interest,Stockholders Equity
2014,1.252415e+09,3.538340e+09,1.032301e+09,2.110884e+09,1.647570e+09


In [102]:
mape_new_concat = np.mean(np.abs((y_test.iloc[:,1:-1] - merged_result) / y_test.iloc[:,1:-1]), axis=1)
mape_new_concat[2014]

0.08623222805710565

In [52]:
pred_concat.loc[li,:]

,Unnamed: 0,Current Assets,Total Non Current Assets,Current Liabilities,Total Non Current Liabilities Net Minority Interest,Stockholders Equity
2014,95,1.264398e+09,3.572197e+09,1.032301e+09,2.110884e+09,1.647570e+09


In [48]:
pred_concat

,Unnamed: 0,Current Assets,Total Non Current Assets,Current Liabilities,Total Non Current Liabilities Net Minority Interest,Stockholders Equity
1078,0,3.173491e+09,8.965812e+09,2.591010e+09,5.298065e+09,4.135165e+09
1957,1,8.072168e+08,4.582965e+08,2.405505e+08,3.207539e+08,7.000295e+08
2694,2,5.154414e+08,1.456251e+09,4.208303e+08,8.605343e+08,6.716404e+08
2002,3,3.155868e+08,8.915942e+08,2.576620e+08,5.268606e+08,4.112179e+08
1213,4,5.762717e+09,1.628111e+10,4.704956e+09,9.620827e+09,7.509108e+09
...,...,...,...,...,...,...
3129,759,9.905800e+08,2.798580e+09,8.087504e+08,1.653736e+09,1.290758e+09
2287,760,3.479171e+08,9.829454e+08,2.840579e+08,5.808428e+08,4.533475e+08
2567,761,8.718634e+08,2.463196e+09,7.118220e+08,1.455552e+09,1.136073e+09
700,762,1.890381e+09,5.340786e+09,1.543389e+09,3.155977e+09,2.463265e+09


In [ ]:
y_test.loc[y_test['new_stock']==stocks_with_two_occurrences[5]]

,Unnamed: 0,Current Assets,Total Non Current Assets,Current Liabilities,Total Non Current Liabilities Net Minority Interest,Stockholders Equity,new_stock
3757,2022-12-31,434982000.0,52947000.0,12584000.0,28352000.0,446993000.0,INTRA
3758,2021-12-31,411699000.0,127440000.0,13423000.0,22800000.0,502916000.0,INTRA


In [38]:
li=y_test.loc[y_test['new_stock']=='INTRA'].index
pred.loc[li,:]

,Unnamed: 0,0,1,2,3,4
3757,131,64159644.0,187391840.0,53602772.0,110226940.0,84805370.0
3758,567,29202950.0,85292360.0,24397706.0,50170370.0,38599930.0


In [39]:
pred_cancat.loc[li,:]

,Unnamed: 0,0,1,2,3,4
3757,131,996534500.0,608989760.0,213448700.0,510227070.0,661591500.0
3758,567,997842560.0,551547260.0,210430780.0,493735870.0,643713150.0


In [59]:
for i in stocks_with_two_occurrences.index:
    li=y_test.loc[y_test['new_stock']==i].index
    # y_test = y_test.apply(pd.to_numeric, errors='coerce')
    # pred = pred.apply(pd.to_numeric, errors='coerce')
    mapsum=abs(y_test.loc[y_test['new_stock']==i]-pred.loc[li,:])

In [61]:
i

'SCL'

In [60]:
y_test.loc[y_test['new_stock']==i]

,Unnamed: 0,Current Assets,Total Non Current Assets,Current Liabilities,Total Non Current Liabilities Net Minority Interest,Stockholders Equity,new_stock


In [58]:
y_test.loc[y_test['new_stock']==i]

,Unnamed: 0,Current Assets,Total Non Current Assets,Current Liabilities,Total Non Current Liabilities Net Minority Interest,Stockholders Equity,new_stock


In [54]:
mapsum

,0,1,2,3,4,Current Assets,Current Liabilities,Stockholders Equity,Total Non Current Assets,Total Non Current Liabilities Net Minority Interest,Unnamed: 0,new_stock


In [51]:
y_test.loc[y_test['new_stock']==i]

,Unnamed: 0,Current Assets,Total Non Current Assets,Current Liabilities,Total Non Current Liabilities Net Minority Interest,Stockholders Equity,new_stock
2891,2022-12-31,353932000.0,921518000.0,197887000.0,200049000.0,874276000.0,VSTO
2892,2021-12-31,426638000.0,976758000.0,263201000.0,231735000.0,903004000.0,VSTO


In [52]:
pred.loc[li,:]

,Unnamed: 0,0,1,2,3,4
2891,339,322039460.0,940583940.0,269051260.0,553271800.0,425662430.0
2892,707,308596800.0,901314500.0,257820640.0,530173340.0,407892200.0


In [46]:
i

2

In [48]:
stocks_with_two_occurrences.index

Index(['VSTO', 'CRWD', 'SW', 'IRDM', 'INTRA', 'JABIL', 'BEACH', 'CARS', 'SLP',
       'RSG',
       ...
       'DAL', 'PAG', 'EBF', 'OII', 'MELI', 'AO', 'AXP', 'VRTV', 'FAIR', 'SCL'],
      dtype='object', name='new_stock', length=123)